# Обработка выгруженных статей

In [71]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.metrics import log_loss

In [129]:
def monthlength(month,year):
    if year % 4 == 0:
         VisYear = 29
    else:
         VisYear = 28
    return [31,VisYear,31,30,31,30,31,31,30,31,30,31][month]

Функция, подсчитывающая частоту каждого слова в статье и выдающая словарь {слово : частота}, а после в DataFrame. Соберем все статьи за конкретный день в одну статью, так как нам необходимо оценить направленность политики в течение дня, и будем работать с суперстатьями.

In [130]:
def Freq(text):
    text = text.replace(",",'').replace(".",'').replace("-"," ")
    tx_new = text.split()
    frequencies = {}
    for item in tx_new:
        if item in frequencies:
            frequencies[item][0]+=1
        else:
            frequencies[item]=[1]
    return(frequencies)

def DF(text):
    #Преообразуем выборку в DataFrame 
    df = pd.DataFrame(Freq(text))
    return(df)

Функция, делающая DataFrame по одному дню какого-то месяца какого-то года.

In [131]:
def DayFrame(day,mouth,year):
    way = "C:/Users/zero/Desktop/ved_dat/" + str(year) + "/" + str(mouth) + "." +str(year) + "/"
    hh = pickle.load(open(way + str(day)+".txt", "rb"))
    text = [hh[i]['text'] for i in range(len(hh))]
    daytext = ''
    for k in text:
        daytext = daytext + " " + k
    initial = DF(daytext)
    return(initial)

DataFrame за месяц

In [132]:
def MouthFrame(mouth,year):
    way = "C:/Users/zero/Desktop/ved_dat/"+ str(year) + "/" + str(mouth) + "." + str(year) + "/"
    tt = [len(pickle.load(open(way + str(hh)+".txt", "rb"))) for hh in range(1,monthlength(mouth-1,year)+1)]
    exist = [i for i in range(len(tt)) if tt[i]>0]
    k = DayFrame(exist[0]+1,mouth,year)
    for m in exist:
        k = k.append(DayFrame(m+1,mouth,year))    
    return(k)

In [133]:
begin = MouthFrame(1,2015)
for item in range(2,13):
    begin=begin.append(MouthFrame(item,2015))

В полученном выше DataFrame есть несколько проблем, с каждой из которых мы ниже будем бороться.

1) Полиморфизм. Каждое слово имеет кучу различных падежей, чисел, склонений. Такие случаи надо как-то унифицировать. Существует два способа бороться с этой проблемой: стэминг (Stemming) и Лемматизация (Lemmatization). Стэминг подразумевает отсечение всех приставок, окончаний и суффиксов. Главный недостаток этого способа в том, что он никогда не переведет слово люди в слово человек. Лемматизация подразумевает использование заранее созданного словаря для перевода всех слов в нормальную форму. Мы для решения этой проблемы будем использовать лематизацию, реализованную для русского языка в пакете pymorthy2.

2) Стоп слова. Зашкаливающее количество предлогов в каждой статье. Можно искусственно вести лист из стоп слов и отскать их с помощью этого листа. Однако мы поступим иначе и просто отсеем слова, встречающиеся более чем в 90-95% статей.

3) Редкие слова. Из-за огромного коилчества редких слов, имеющаяся у нас матрица разряжена и для оценки 10 с лишним тысяч коэффициентов имеется лишь около 1000 наблюдений. Придется выкидывать редкие слова для того, чтобы этот дисбаланс преодалеть.

4) Орфографические ошибки в статьях.

Для решения первой проблемы вошьем методы pymorphy2 в функции, написанные выше и получим новый DataFrame.

Установим пакет через pip, вбив соответствующую команду в командную строку.

In [135]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

Добавим pymorthy2 в нашу функцию и заново задатафреймим статейки!

In [136]:
def Freq(text):
    text = text.replace(",",'').replace(".",'').replace("-"," ")
    tx_new = text.split()
    tx_pm = [morph.parse(item)[0].normal_form for item in tx_new]     
    frequencies = {}
    for item in tx_pm:
        if item in frequencies:
            frequencies[item][0]+=1
        else:
            frequencies[item]=[1]
    return(frequencies)

def DF(text):
    #Преообразуем выборку в DataFrame 
    df = pd.DataFrame(Freq(text))
    return(df)

In [137]:
pmdf2015 = MouthFrame(1,2015)
for item in range(2,13):
    pmdf2015=pmdf2015.append(MouthFrame(item,2015))

Аналогично соберём DataFrame за 2014, январь - апрель 2016 года и июль - декабрь 2013 года. 

In [139]:
pmdf2014 = MouthFrame(1,2014)
for item in range(2,13):
    pmdf2014=pmdf2014.append(MouthFrame(item,2014))

In [140]:
pmdf2013 = MouthFrame(7,2013)
for item in range(8,13):
    pmdf2013=pmdf2013.append(MouthFrame(item,2013))

In [141]:
pmdf2016 = MouthFrame(1,2016)
for item in range(2,5):
    pmdf2016=pmdf2016.append(MouthFrame(item,2016))

Мы собрали четыре DataFrame. Объединим их и приведем в порядок.

In [142]:
bigdf = pmdf2013
bigdf = bigdf.append(pmdf2014)
bigdf = bigdf.append(pmdf2015)
bigdf = bigdf.append(pmdf2016)

Всего имеем в рассмотрении 706 дней, 15492 слова. При этом классифицированы из них 266. Очистим DataFrame от стоп слов и редких слов так, чтобы наблюдений хватило для оценки коэффициентов в логистической регрессии. Заменим все NaN в DataFrame на нули.

In [145]:
bigdf.fillna(0, inplace=True)
dim = bigdf.shape

Проанализируем насколько часто встречаются в нашей выборке отдельные слова.

In [149]:
def WordCount(df,column):
    name = df.columns
    r = np.array(df[name[column]])
    k = 0
    for item in r:
        if item !=0:
            k=k+1
    return(k/dim[0]*100)    

cowo = np.array([WordCount(bigdf,jtem) for jtem in range(dim[1])])

In [150]:
name = bigdf.columns
sortname = np.argsort(cowo)
print(name[sortname[-32:]])
print(cowo[sortname[-32:]])

Index(['мочь', 'от', 'тот', 'для', 'регулятор', 'а', 'рубль', 'до', 'который',
       'за', 'из', 'к', 'он', 'о', 'банк', 'что', 'банка', 'не', 'россия',
       'это', 'год', 'быть', 'по', 'с', 'на', 'цб', 'и', 'в', 'определение',
       'приватный', 'просмотр', 'режим'],
      dtype='object')
[  81.72804533   82.57790368   83.28611898   83.42776204   84.70254958
   85.12747875   85.83569405   87.11048159   87.81869688   88.52691218
   88.52691218   89.80169972   89.80169972   90.79320113   90.79320113
   91.78470255   91.92634561   92.63456091   93.20113314   93.62606232
   94.0509915    94.47592068   96.31728045   97.02549575   99.00849858
   99.15014164   99.15014164   99.71671388  100.          100.          100.
  100.        ]


Посмотрим на самые редкие слова. Всего в одну статью. попадает 6053 слова. 

In [152]:
print(name[sortname[:20]])
print(cowo[sortname[:20]])

def WF(how):
    l=0
    for item in cowo:
        if item == how/dim[0]*100:
            l=l+1
    return(l)

print([WF(jtem) for jtem in range(1,30)])

Index(['навнутренний', 'натиксис', 'натакий', 'насыщенный', 'насыпь',
       'насущный', 'настроенность', 'финаму', 'настраивать', 'настоятельно',
       'настороженность', 'настороженно', 'настепень', 'насовсем', 'наслучай',
       'наследница', 'наследить', 'наследие', 'финансито', 'финарс'],
      dtype='object')
[ 0.14164306  0.14164306  0.14164306  0.14164306  0.14164306  0.14164306
  0.14164306  0.14164306  0.14164306  0.14164306  0.14164306  0.14164306
  0.14164306  0.14164306  0.14164306  0.14164306  0.14164306  0.14164306
  0.14164306  0.14164306]
[6053, 2155, 1085, 728, 481, 398, 320, 278, 219, 174, 183, 128, 119, 122, 99, 94, 95, 84, 80, 83, 63, 53, 70, 57, 60, 50, 52, 50, 50]


Слов с частотой менее 1% всего будет 11220 штук. С частотой менее 0.5% 9293 слова. Нам необходимо получить количество коэффициентов, которое не привысило бы количество наблюдений, то есть было бы меньше чем 266. Чтобы получить ровно 266 коэффициентов нужно выкинуть 15226 слова. Если отсеять слова, встречающиеся менее чем в 5% статей, то будет выброшено 13716 слов. Необходимо будет оценить 757 коэффициентов по 958 наблюдениям. Остановимся пока что на этой цифре.

In [163]:
l = 0
for item in cowo:
    if item < 40:
        l=l+1
l

15316

Выбрасываем из DataFrame редкие слова. Их выброс заодно решит проблему орфографических ошибок. Также выбрасываем слова, встречающиеся чаще, чем в 70% статей. Отметим, что слово а встречается с частотой 53%.

In [58]:
bigdf = bigdf.drop(name[sortname[:14736]], axis=1)
bigdf.drop(name[sortname[-19:]],axis=1,inplace=True)

Выделим тренировочную и тестовую выборкии. В тренировочную выборку войдёт 2015 год, в тестовую 2016 год.

In [61]:
dim3 = pmdf2013.shape
dim4 = pmdf2014.shape
dim5 = pmdf2015.shape
dim6 = pmdf2016.shape

In [100]:
train0 = dim3[0] + dim4[0]
train1 = dim[0] - dim6[0]

In [110]:
train = pd.DataFrame(np.array(bigdf)[train0:train1])
test = pd.DataFrame(np.array(bigdf)[train1:dim[0]])

In [117]:
myaw = bigdf.columns
train.columns = myaw
test.columns = myaw

Подгрузим ответы для статей из тренировочной выбоки и ответы для статей из тестовой выборки.

In [122]:
len(a)

930

In [123]:
len(train)

958

# Приятные мелочи

Рисунок с гистограммой для презентации.

In [95]:
plt.hist(cowo[sortname[14700:15492]], bins=100)
plt.show()

# Классификация по заголовкам

In [192]:
def DayFrame(day,mouth,year):
    way = "C:/Users/zero/Desktop/ved_dat/" + str(year) + "/" + str(mouth) + "." +str(year) + "/"
    hh = pickle.load(open(way + str(day)+".txt", "rb"))
    text = [hh[i]['title'] for i in range(len(hh))]
    daytext = ''
    for k in text:
        daytext = daytext + " " + k
    initial = DF(daytext)
    return(initial)

In [193]:
pmdf2015 = MouthFrame(1,2015)
for item in range(2,13):
    pmdf2015=pmdf2015.append(MouthFrame(item,2015))
    
pmdf2014 = MouthFrame(1,2014)
for item in range(2,13):
    pmdf2014=pmdf2014.append(MouthFrame(item,2014))

pmdf2013 = MouthFrame(7,2013)
for item in range(8,13):
    pmdf2013=pmdf2013.append(MouthFrame(item,2013))
    
pmdf2016 = MouthFrame(1,2016)
for item in range(2,5):
    pmdf2016=pmdf2016.append(MouthFrame(item,2016))
    
bigdf = pmdf2013
bigdf = bigdf.append(pmdf2014)
bigdf = bigdf.append(pmdf2015)
bigdf = bigdf.append(pmdf2016)

bigdf.fillna(0, inplace=True)
dim = bigdf.shape

Частые слова

In [194]:
def WordCount(df,column):
    name = df.columns
    r = np.array(df[name[column]])
    k = 0
    for item in r:
        if item !=0:
            k=k+1
    return(k/dim[0]*100)    

cowo = np.array([WordCount(bigdf,jtem) for jtem in range(dim[1])])

name = bigdf.columns
sortname = np.argsort(cowo)
print(name[sortname[-32:]])
print(cowo[sortname[-32:]])

Index(['коридор', 'кредит', 'набиуллин', 'рынок', 'из', 'к', 'от', 'мочь', 'о',
       'курс', 'до', 'валютный', 'для', 'отозвать', 'миллиард', 'за', 'у',
       'ставка', 'не', 'лицензия', 'по', 'с', 'год', 'и', 'центробанк',
       'россия', 'рубль', 'банка', 'банк', 'на', 'в', 'цб'],
      dtype='object')
[ 11.17647059  11.32352941  12.05882353  12.35294118  12.5         13.08823529
  13.08823529  14.41176471  15.          15.          15.58823529
  15.58823529  15.58823529  16.91176471  19.55882353  20.44117647
  21.17647059  22.05882353  23.67647059  23.82352941  24.26470588
  25.14705882  25.29411765  25.88235294  26.32352941  27.5         31.32352941
  32.20588235  35.58823529  45.          56.17647059  94.55882353]


Редкие слова

In [195]:
print(name[sortname[:20]])
print(cowo[sortname[:20]])

def WF(how):
    l=0
    for item in cowo:
        if item == how/dim[0]*100:
            l=l+1
    return(l)

print([WF(jtem) for jtem in range(1,30)])

Index(['протест', 'ловушка', 'подконтрольный', 'стагфляционный', 'лишение',
       'лишать', 'личный', 'простой', 'подмосковный', 'поднадзорный',
       'листинг', 'лист', 'линк', 'лимитировать', 'сталин', 'поднимать',
       'стандартный', 'ремонт', 'логотип', 'поделиться'],
      dtype='object')
[ 0.14705882  0.14705882  0.14705882  0.14705882  0.14705882  0.14705882
  0.14705882  0.14705882  0.14705882  0.14705882  0.14705882  0.14705882
  0.14705882  0.14705882  0.14705882  0.14705882  0.14705882  0.14705882
  0.14705882  0.14705882]
[1352, 484, 222, 135, 96, 66, 55, 49, 29, 26, 25, 36, 21, 16, 9, 10, 6, 21, 8, 13, 10, 7, 6, 5, 6, 9, 5, 0, 5]


In [197]:
l = 0
for item in cowo:
    if item < 2:
        l=l+1
l

2596

Всего 2838 коэффициентов. Наблюдений всего 266. Отсечем слова, встречающиеся менее чем в 2% статей и более чем в 50% статей. После редукции требуется оценить 191 коэффициент.

In [198]:
bigdf = bigdf.drop(name[sortname[:2645]], axis=1)
bigdf.drop(name[sortname[-2:]],axis=1,inplace=True)
bigdf.shape

(680, 157)

Выделим тренировочную и тестовую выборкии. В тренировочную выборку войдёт 2015 год, в тестовую 2016 год.

In [199]:
dim3 = pmdf2013.shape
dim4 = pmdf2014.shape
dim5 = pmdf2015.shape
dim6 = pmdf2016.shape

train0 = dim3[0] + dim4[0]
train1 = dim[0] - dim6[0]

train = pd.DataFrame(np.array(bigdf)[train0:train1])
test = pd.DataFrame(np.array(bigdf)[train1:dim[0]])

myaw = bigdf.columns
train.columns = myaw
test.columns = myaw

Подгрузим ответы для статей из тренировочной выбоки и ответы для статей из тестовой выборки.

In [200]:
train.shape

(240, 157)

In [206]:
mouth=9
way = "C:/Users/zero/Desktop/ved_dat/"+ str(2015) + "/" + str(mouth) + "." + str(2015) + "/"
tt = [len(pickle.load(open(way + str(hh)+".txt", "rb"))) for hh in range(1,monthlength(mouth-1,2015)+1)]
exist = [i+1 for i in range(len(tt)) if tt[i]>0]
exist

[1,
 2,
 3,
 4,
 8,
 9,
 10,
 11,
 14,
 15,
 16,
 17,
 18,
 20,
 21,
 22,
 23,
 24,
 25,
 28,
 29,
 30]